# Loading libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.feature_selection import SelectFromModel
import xgboost as xg


import missingno as msno 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import KMeans, DBSCAN
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet
from  sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller
from numpy import log
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import multiprocessing
from sklearn.model_selection import KFold
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_validate



from sklearn.preprocessing import OrdinalEncoder



%matplotlib inline
sns.set_style("darkgrid")
from sklearn.metrics import mean_squared_error

# Importing the dataset

In [3]:
community_mesos = pd.read_csv('community_mesos.csv').set_index('month_year')
df_prova = pd.read_csv('df_prova.csv')
df = pd.read_csv('df.csv')

df['month_year'] = pd.to_datetime(df.Dates).dt.to_period('M')
df['Dates'] = pd.to_datetime(df['Date']).dt.date
df['Time'] = pd.to_datetime(df['Date']).dt.time

selected_types = ['CRIMINAL SEXUAL ASSAULT', 'SEX OFFENSE']
descriptions = ['PUBLIC INDECENCY', 'SEXUAL EXPLOITATION OF A CHILD', 
       'INDECENT SOLICITATION/CHILD', 'CRIMINAL TRANSMISSION OF HIV',
       'INDECENT SOLICITATION OF A CHILD', 'FORNICATION', 'BIGAMY',
        'MARRYING A BIGAMIST']
df.Dates = pd.to_datetime(df.Dates)
df_prova = df[(df['Primary Type'] == 'CRIMINAL SEXUAL ASSAULT') | (df['Primary Type'] == 'SEX OFFENSE')]
for description in descriptions:
    df_prova = df_prova[df_prova['Description'] != description]

# Setting main dates

In [4]:
date ={}
date['date_start_train'] = '2001-01-01'
date['date_end_train_mesos'] = '2021-12'

date['date_start_test_mesos'] = '2022-01'
date['date_end_test_mesos'] = '2022-12'

In [5]:
# !pip install gluonts
# !pip install mxnet

In [6]:
%matplotlib inline
import plotly.express as px
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as sm
from statsmodels.tsa.stattools import kpss ,  adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gluonts
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer import Trainer
import matplotlib.pyplot  as plt
import pandas as pd
import matplotlib as mpl

from mxnet import gpu
import mxnet as mx


from gluonts.evaluation import Evaluator


In [7]:
community_mesos = community_mesos.drop(columns=['agressions_community_2_month_avg', 'agressions_community_6_month_avg', 'agressions_lag_1',  'agressions_lag_2', 'agressions_lag_3', 'agressions_lag_4'])

In [8]:
community_mesos.keys()

Index(['Community Area', 'Agressions', 'New Years', 'month', 'primavera',
       'estiu', 'tardor', 'hivern', 'year_2002', 'year_2003', 'year_2004',
       'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009',
       'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014',
       'year_2015', 'year_2016', 'year_2017', 'year_2018', 'year_2019',
       'year_2020', 'year_2021', 'year_2022', 'quarter_2', 'quarter_3',
       'quarter_4', 'trend', 'sin(1,freq=A-DEC)', 'cos(1,freq=A-DEC)',
       'sin(2,freq=A-DEC)', 'cos(2,freq=A-DEC)', 'sin(3,freq=A-DEC)',
       'cos(3,freq=A-DEC)', 'sin(4,freq=A-DEC)', 'cos(4,freq=A-DEC)',
       'sin(5,freq=A-DEC)', 'cos(5,freq=A-DEC)', 'cos(1,freq=M)',
       'sin(1,freq=W-SUN)', 'cos(1,freq=W-SUN)', 'sin(2,freq=W-SUN)',
       'cos(2,freq=W-SUN)', 'cos(3,freq=W-SUN)', 'sin(4,freq=W-SUN)',
       'Poverty', 'Foreign born', 'HS Grads', 'BA Grads'],
      dtype='object')

In [9]:

def add_header_keys(df, keys, header):
    map_ = {k: header + k for k in keys}
    return df.rename(columns=map_)


def split_dynamic_stat_data(df, keys=None, thresh=0.05):
    if keys is None:
        keys = [k for k in df.keys() if k not in ['Agressions', 'Community Area', 'ds']]
    dynamic = []
    unique_codes = np.unique(df['Community Area'])
    limit = int(thresh * len(unique_codes))
    counts = {k: [] for k in keys}
    for code in unique_codes:
        code_df = df[df['Community Area'] == code]
        for k in keys:
            if len(np.unique(code_df[k])) > 1:
                #print(
                    #f'For key {k}, code {code} has voted that its dynamical. Unique values are {np.unique(code_df[k])}')
                counts[k].append(code)
                if len(counts[k]) >= limit:
                    #print(f'For key {k}, We have found its dynamical after codes {counts[k]} voted that its dynamical')
                    dynamic.append(k)
                    keys.remove(k)
    static = [k for k in keys]

    return dynamic, static



def split_real_categorical_data(df, thresh=0.1, keys=[], isdynamical=True, auto=False):
    """
    To decide if a variable is categorical or not, we can do the following.
    Take the length of all the values in a series and compare it with the unique values of the series.
    If the unique values is less than 10% of the length of the full series, it means that the variable is categorical.
    Else, the variable is dynamic.
    This nemotecnic rule should work pretty good.
    """

    if len(keys) == 0 and auto:
        keys = [k for k in df.keys() if k not in ['Agressions', 'Community Area', 'ds']]
    elif len(keys) == 0:
        return [], []

    if isdynamical:
        """
        this will not work for statical data... but it is the most pure way to do it.
        """
        votes = {k: 0 for k in keys}
        for code in np.unique(df['Community Area']):
            code_df = df[df['Community Area'] == code]
            for k in keys:
                if len(code_df[k]) > 1:
                    if len(np.unique(code_df[k])) < thresh * len(code_df[k]) or len(np.unique(code_df[k])) == 1:
                        votes[k] -= 1
                    else:
                        votes[k] += 1.5

        categorical = []
        real = []
        for k in votes.keys():
            if votes[k] >= 0:
                real.append(k)
            else:
                categorical.append(k)

    else:
        categorical = []
        for k in keys:
            if len(np.unique(df[k])) < thresh * len(df):
                categorical.append(k)
                keys.remove(k)
        real = keys

    print(f'Real keys before formatting {real}')
    print(f'Categorical keys before formatting {categorical}')
    print()
    for k in real:
        if isinstance(np.array(df[k])[0], str):
            categorical.append(k)
            real.remove(k)
    for k in categorical:
        if isinstance(np.array(df[k])[0], float):
            real.append(k)
            categorical.remove(k)
    print(f'Real keys after formatting {real}')
    print(f'Categorical keys after formatting {categorical}')
    return categorical, real



def split_features(df, thresh=0.1):
    dyn, stat = split_dynamic_stat_data(df)
    dyn_cat, dyn_re = split_real_categorical_data(df, thresh=thresh, keys=dyn)
    stat_cat, stat_re = split_real_categorical_data(df, thresh=thresh, keys=stat)
    return dyn_cat, dyn_re, stat_cat, stat_re

def generate_labelled_exogenous_df(df, one_hot=True, thresh=0.1,ignore_keys = [['Community Area','Agressions']]):
    df = df.fillna(0)
    dyn_cat, dyn_re, stat_cat, stat_re = split_features(df, thresh=thresh)
    print('Dynamical categorical features:')
    for k in dyn_cat:
        print(k)
    print('\n\n')
    print('Dynamical real features:')
    for k in dyn_re:
        print(k)
    print('\n\n')
    print('Statical categorical features:')
    for k in stat_cat:
        print(k)
    print('\n\n')
    print('Statical real features:')
    for k in stat_re:
        print(k)
    print('\n\n')
    new_df = df.copy()
    new_df = add_header_keys(new_df, dyn_cat, 'dyn_cat_')
    new_df = add_header_keys(new_df, dyn_re, 'dyn_re_')
    new_df = add_header_keys(new_df, stat_cat, 'stat_cat_')
    new_df = add_header_keys(new_df, stat_re, 'stat_re_')

    ordinal_encoder = None

    if one_hot:
        cat_dyn_keys = [k for k in new_df.keys() if 'dyn_cat_' in k]
        for k in cat_dyn_keys:
            tmp = pd.get_dummies(new_df[k], prefix=k)
            for new_k in tmp.keys():
                new_k_re = new_k.replace('dyn_cat_', 'dyn_re_OneHot_')
                new_df[new_k_re] = tmp[new_k].copy()
        new_df = new_df.drop(cat_dyn_keys, axis=1)

        cat_sat_keys = [k for k in new_df.keys() if 'stat_cat_' in k]
        features = new_df[cat_sat_keys].to_numpy()
        ordinal_encoder = OrdinalEncoder().fit(features)
        encoded_features = ordinal_encoder.transform(features)
        for i, k in enumerate(cat_sat_keys):
            new_df[k] = encoded_features[:, i]

    return new_df, ordinal_encoder

In [10]:
dd, encod = generate_labelled_exogenous_df(community_mesos, one_hot = False)

Real keys before formatting ['trend']
Categorical keys before formatting ['New Years', 'primavera', 'tardor', 'year_2002', 'year_2004', 'year_2006', 'year_2008', 'year_2010', 'year_2012', 'year_2014', 'year_2016', 'year_2018', 'year_2020', 'year_2022', 'quarter_3', 'cos(1,freq=A-DEC)', 'cos(2,freq=A-DEC)', 'cos(3,freq=A-DEC)', 'cos(4,freq=A-DEC)', 'cos(5,freq=A-DEC)', 'sin(1,freq=W-SUN)', 'sin(2,freq=W-SUN)', 'cos(3,freq=W-SUN)', 'Poverty', 'HS Grads', 'month', 'hivern', 'year_2005', 'year_2009', 'year_2013', 'year_2017', 'year_2021', 'quarter_4', 'sin(2,freq=A-DEC)', 'sin(4,freq=A-DEC)', 'cos(1,freq=W-SUN)', 'sin(4,freq=W-SUN)', 'BA Grads', 'estiu', 'year_2007', 'year_2015', 'quarter_2', 'sin(3,freq=A-DEC)', 'cos(2,freq=W-SUN)', 'year_2003', 'year_2019', 'sin(5,freq=A-DEC)', 'Foreign born', 'year_2011', 'sin(1,freq=A-DEC)']

Real keys after formatting ['trend', 'cos(1,freq=A-DEC)', 'cos(3,freq=A-DEC)', 'cos(5,freq=A-DEC)', 'sin(2,freq=W-SUN)', 'Poverty', 'sin(2,freq=A-DEC)', 'cos(1,fr

In [11]:
dd

,Community Area,Agressions,dyn_cat_New Years,dyn_cat_month,dyn_cat_primavera,dyn_cat_estiu,dyn_cat_tardor,dyn_cat_hivern,dyn_cat_year_2002,dyn_cat_year_2003,...,"dyn_cat_sin(1,freq=W-SUN)","dyn_re_cos(1,freq=W-SUN)","dyn_re_sin(2,freq=W-SUN)","dyn_cat_cos(2,freq=W-SUN)","dyn_cat_cos(3,freq=W-SUN)","dyn_cat_sin(4,freq=W-SUN)",dyn_re_Poverty,dyn_cat_Foreign born,dyn_cat_HS Grads,dyn_re_BA Grads
month_year,,,,,,,,,,,,,,,,,,,,,
2001-01,1.0,1.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.213,0.338,0.746,0.320
2001-01,2.0,0.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.143,0.456,0.804,0.356
2001-01,3.0,1.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.249,0.330,0.764,0.395
2001-01,4.0,0.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.114,0.386,0.785,0.381
2001-01,5.0,0.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.086,0.145,0.850,0.481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,73.0,0.0,0,12,0,0,0,1,0,0,...,0.433884,-0.900969,-0.781831,0.62349,-0.222521,-0.974928,0.200,0.008,0.886,0.262
2022-12,74.0,0.0,0,12,0,0,0,1,0,0,...,0.433884,-0.900969,-0.781831,0.62349,-0.222521,-0.974928,0.046,0.038,0.953,0.401
2022-12,75.0,0.0,0,12,0,0,0,1,0,0,...,0.433884,-0.900969,-0.781831,0.62349,-0.222521,-0.974928,0.115,0.035,0.937,0.393


In [12]:
cols_dyn_re = dd.columns[dd.columns.str.match(r'dyn_re')]
cols_stat_cat = dd.columns[dd.columns.str.match(r'stat_cat')]
cols_dyn_cat = dd.columns[dd.columns.str.match(r'dyn_cat')]
cols_stat_re = dd.columns[dd.columns.str.match(r'stat_re')]

In [13]:
len(cols_dyn_re),len(cols_stat_cat),len(cols_dyn_cat),len(cols_stat_re)

(12, 0, 39, 1)

In [14]:
cols_dyn_re = cols_dyn_re.tolist() + cols_dyn_cat.tolist()

In [15]:
cols_stat_cat.tolist()

[]

In [16]:
cardinality = dd[cols_stat_cat.tolist()].max()+1

In [17]:
cardinality = cardinality.astype(int).tolist()

In [18]:
dd.keys()

Index(['Community Area', 'Agressions', 'dyn_cat_New Years', 'dyn_cat_month',
       'dyn_cat_primavera', 'dyn_cat_estiu', 'dyn_cat_tardor',
       'dyn_cat_hivern', 'dyn_cat_year_2002', 'dyn_cat_year_2003',
       'dyn_cat_year_2004', 'dyn_cat_year_2005', 'dyn_cat_year_2006',
       'dyn_cat_year_2007', 'dyn_cat_year_2008', 'dyn_cat_year_2009',
       'dyn_cat_year_2010', 'dyn_cat_year_2011', 'dyn_cat_year_2012',
       'dyn_cat_year_2013', 'dyn_cat_year_2014', 'dyn_cat_year_2015',
       'dyn_cat_year_2016', 'dyn_cat_year_2017', 'dyn_cat_year_2018',
       'dyn_cat_year_2019', 'dyn_cat_year_2020', 'dyn_cat_year_2021',
       'dyn_cat_year_2022', 'dyn_cat_quarter_2', 'dyn_cat_quarter_3',
       'dyn_cat_quarter_4', 'dyn_re_trend', 'dyn_re_sin(1,freq=A-DEC)',
       'dyn_re_cos(1,freq=A-DEC)', 'dyn_re_sin(2,freq=A-DEC)',
       'dyn_cat_cos(2,freq=A-DEC)', 'dyn_re_sin(3,freq=A-DEC)',
       'dyn_re_cos(3,freq=A-DEC)', 'dyn_cat_sin(4,freq=A-DEC)',
       'dyn_cat_cos(4,freq=A-DEC)', 'dyn

In [19]:
dd

,Community Area,Agressions,dyn_cat_New Years,dyn_cat_month,dyn_cat_primavera,dyn_cat_estiu,dyn_cat_tardor,dyn_cat_hivern,dyn_cat_year_2002,dyn_cat_year_2003,...,"dyn_cat_sin(1,freq=W-SUN)","dyn_re_cos(1,freq=W-SUN)","dyn_re_sin(2,freq=W-SUN)","dyn_cat_cos(2,freq=W-SUN)","dyn_cat_cos(3,freq=W-SUN)","dyn_cat_sin(4,freq=W-SUN)",dyn_re_Poverty,dyn_cat_Foreign born,dyn_cat_HS Grads,dyn_re_BA Grads
month_year,,,,,,,,,,,,,,,,,,,,,
2001-01,1.0,1.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.213,0.338,0.746,0.320
2001-01,2.0,0.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.143,0.456,0.804,0.356
2001-01,3.0,1.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.249,0.330,0.764,0.395
2001-01,4.0,0.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.114,0.386,0.785,0.381
2001-01,5.0,0.0,1,1,0,0,0,1,0,0,...,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,0.086,0.145,0.850,0.481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,73.0,0.0,0,12,0,0,0,1,0,0,...,0.433884,-0.900969,-0.781831,0.62349,-0.222521,-0.974928,0.200,0.008,0.886,0.262
2022-12,74.0,0.0,0,12,0,0,0,1,0,0,...,0.433884,-0.900969,-0.781831,0.62349,-0.222521,-0.974928,0.046,0.038,0.953,0.401
2022-12,75.0,0.0,0,12,0,0,0,1,0,0,...,0.433884,-0.900969,-0.781831,0.62349,-0.222521,-0.974928,0.115,0.035,0.937,0.393


In [24]:
mx.random.seed(7)
np.random.seed(7)

In [25]:
class DeepAR() : 
    def __init__(self,df) : 

        """ 
        init (self,df) : the constructor of the class

        """
        self.df = df
    
    def prepare_dataset(self,start_index,target_index,target,freq):
        dataset = ListDataset([{"start": start_index, 
                                            "target": self.df[target][start_index:target_index]}], freq = freq)
        return dataset
    
    def prepare_dataset_grouped(self,start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list, groupby_index = 'Community Area'):
        dataset_list = []
        counter = 0
        total = len(self.df[groupby_index].unique())
        for item_id,df_item in self.df.groupby(groupby_index,as_index = False):
            counter+=1
            if counter%200==0:
                print (f'{counter}/{total}')
            current_dataset = {"start": start_index, 
                                            "target": self.df[target][start_index:target_index]}

            dynre_list = []
            statcat_list = []
            statre_list = []

            for dyn_re in dyn_re_list:
                dynre_list.append(self.df[dyn_re][start_index:target_index])

            for stat_cat in stat_cat_list:
                statcat_list.append(self.df[stat_cat].iloc[0])

            for stat_re in stat_re_list:
                statre_list.append(self.df[stat_re].iloc[0])


#             current_dataset[FieldName.FEAT_STATIC_CAT] = statcat_list
            current_dataset[FieldName.FEAT_DYNAMIC_REAL] = dynre_list
            current_dataset[FieldName.FEAT_STATIC_REAL] = statre_list

            dataset_list.append(current_dataset)



        dataset = ListDataset(dataset_list, freq = freq)

        return dataset  


    def prepare_train (self,start_index,target_index,target, freq, dyn_re_list, stat_cat_list, stat_re_list) : 
        self.training_data = self.prepare_dataset_grouped(start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list)

    def prepare_test (self,start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list) : 

        self.test_data = self.prepare_dataset_grouped(start_index,target_index,target,freq, dyn_re_list, stat_cat_list, stat_re_list)

    def estimator (self,freq,context_length,prediction_length,num_layers,num_cells,cell_type,epochs,cardinality, use_external = True) : 

        """"
        In order to fix the architecture of the estimator .. num_layers , cell_type etc ...

        """

        self.estimator = DeepAREstimator(freq=freq, 
                                context_length=context_length,  
                                prediction_length=prediction_length,
                                num_layers=num_layers,    
                                num_cells=num_cells,
                                cell_type=cell_type,
                                #trainer=Trainer(epochs=epochs,ctx=gpu())) 
                                trainer=Trainer(epochs=epochs),
                                use_feat_dynamic_real=True*use_external,
                                use_feat_static_cat=False*use_external,
                                use_feat_static_real=True*use_external,
                                distr_output = NegativeBinomialOutput())
#                                 cardinality = cardinality) 

        self.prediction_length = prediction_length

    def train(self) : 

        """
        to do the fit

        """
        self.predictor = self.estimator.train(training_data=self.training_data)

    def evaluate (self,num_samples=100) : 

        """ 
        to do evaluation task

        """

        forecast_it, ts_it = make_evaluation_predictions(
        dataset=self.test_data,  
        predictor=self.predictor,   
        num_samples=num_samples, 
        )
        self.forecasts = list(forecast_it)
        self.tss =  list(ts_it)


    def plot_prob_forecasts(self, index):

        """
        plot the forecasts

        """
        print('hello')
        prediction_intervals = (80.0, 95.0)
        legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]
        fig, ax = plt.subplots(1, 1, figsize=(10, 7))
        self.tss[index].plot(ax=ax) 
        self.forecasts[index].plot(color='g')
        #self.forecasts[index].plot(prediction_intervals=prediction_intervals, color='g')
        plt.grid(which="both")
        plt.legend(legend, loc="upper left")
        plt.show()
         

In [26]:
deepar = DeepAR(dd)
deepar.prepare_train(date['date_start_train'],date['date_end_train_mesos'],'Agressions','d', cols_dyn_re, cols_stat_cat, cols_stat_re)
cardinality = np.array(dd[cols_stat_cat].max()+1)

deepar.estimator(freq = 'd',context_length = 100, prediction_length = 462, num_layers = 2, num_cells = 40,cell_type = "lstm",epochs = 50,cardinality = cardinality.tolist())
deepar.train()

date['date_start_test_mesos'] = '2001-01'

deepar.prepare_test(date['date_start_test_mesos'],date['date_end_test_mesos'],'Agressions','d', cols_dyn_re, cols_stat_cat, cols_stat_re)
deepar.evaluate(num_samples=100)

100%|████████████████████████████████████████████████| 50/50 [01:57<00:00,  2.35s/it, epoch=50/50, avg_epoch_loss=1.93]


In [108]:
def rmsle(y_pred,y):
    return np.sqrt(np.mean((np.log(y+1)-np.log(y_pred+1))**2))

In [27]:
y_pred = deepar.forecasts[0].samples.mean(axis = 0)
y_real = np.array(deepar.tss[0]).reshape(-1)[1:]

In [194]:
warnings.filterwarnings("ignore")

In [28]:
f = df_prova.groupby([df['month_year'], df['Community Area']]).count().unstack(fill_value=0).stack()
f = f.iloc[:, :-24]
f = f.rename(columns = {'ID':'Agressions'})
f= f.reset_index().set_index('month_year')
f = f[f['Community Area'] != 0]
f = f.iloc[:, 1:]
f = f.loc[f.index.year != 2023]
test = f.iloc[-462:]

In [29]:
test['Pred'] = y_pred

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
def sqr_err(y_true, y_pred):
    """

    :param y_true: true values of y
    :param y_pred: predicted values of y
    :return: array of lenght original data containing mean squared error for each predictions
    """
    if len(y_true) != len(y_pred):
        raise IndexError("Mismathced array sizes, you inputted arrays with sizes {} and {}".format(len(y_true),
                                                                                                  len(y_pred)))
    else:
        length = len(y_true)

    sqrerror_out = [(y_pred[i]-y_true[i])**2 for i in range(length)]

    return np.array(sqrerror_out)

In [32]:
test['Mean model'] = test['Agressions'].mean()
test['MAPE'] = np.mean(np.abs((test['Agressions'] - test['Pred'])/test['Agressions']))*100
test['MAPE mean'] = np.mean(np.abs((test['Agressions'] - test['Mean model'])/test['Agressions']))*100
test['SMAPE'] = np.mean(np.abs((test['Pred']-test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['SMAPE mean'] = np.mean(np.abs((test['Mean model'] - test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['MAE'] = np.mean(np.abs(test['Agressions'] - test['Pred']))
test['MAE mean'] = np.mean(np.abs(test['Agressions'] - test['Mean model']))
test['Metrica nomb'] = (test['Agressions'] < test['Pred']).sum()/len(test['Pred'])*100
test['Metrica nomb mean'] = (test['Agressions'] < test['Mean model']).sum()/len(test['Pred'])*100
test['RMSE'] = np.sqrt(mean_squared_error(test['Agressions'], test['Pred']))
test['RMSE mean'] = np.sqrt(mean_squared_error(test['Agressions'], test['Mean model']))
test['MSE'] = sqr_err(test['Agressions'], test['Pred'])
test

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,Agressions,Pred,Mean model,MAPE,MAPE mean,SMAPE,SMAPE mean,MAE,MAE mean,Metrica nomb,Metrica nomb mean,RMSE,RMSE mean,MSE
month_year,,,,,,,,,,,,,,
2022-07,2,3.59,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,2.5281
2022-07,2,3.99,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,3.9601
2022-07,6,5.49,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,0.2601
2022-07,3,3.48,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,0.2304
2022-07,1,2.37,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,1.8769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,1,1.61,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,0.3721
2022-12,0,1.84,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,3.3856
2022-12,1,2.11,2.692641,inf,inf,93.034126,105.158725,2.128896,2.162572,59.95671,60.17316,3.044263,3.03242,1.2321


In [ ]:
################# Predicció anual ################333

In [35]:
deepar = DeepAR(dd)
deepar.prepare_train(date['date_start_train'],date['date_end_train_mesos'],'Agressions','d', cols_dyn_re, cols_stat_cat, cols_stat_re)
cardinality = np.array(dd[cols_stat_cat].max()+1)

deepar.estimator(freq = 'd',context_length = 100, prediction_length = 924, num_layers = 2, num_cells = 40,cell_type = "gru",epochs = 50,cardinality = cardinality.tolist())
deepar.train()

date['date_start_test_mesos'] = '2001-01'

deepar.prepare_test(date['date_start_test_mesos'],date['date_end_test_mesos'],'Agressions','d', cols_dyn_re, cols_stat_cat, cols_stat_re)
deepar.evaluate(num_samples=100)

100%|████████████████████████████████████████████████| 50/50 [01:09<00:00,  1.38s/it, epoch=50/50, avg_epoch_loss=1.95]


In [36]:
y_pred = deepar.forecasts[0].samples.mean(axis = 0)
y_real = np.array(deepar.tss[0]).reshape(-1)[1:] #EL primer punt no el prediu! El fa servir com entrada inicial

f = df_prova.groupby([df['month_year'], df['Community Area']]).count().unstack(fill_value=0).stack()
f = f.iloc[:, :-24]
f = f.rename(columns = {'ID':'Agressions'})
f= f.reset_index().set_index('month_year')
f = f[f['Community Area'] != 0]
f = f.iloc[:, 1:]
f = f.loc[f.index.year != 2023]
test = f.iloc[-924:]

test['Pred'] = y_pred

test['Mean model'] = test['Agressions'].mean()
test['MAPE'] = np.mean(np.abs((test['Agressions'] - test['Pred'])/test['Agressions']))*100
test['MAPE mean'] = np.mean(np.abs((test['Agressions'] - test['Mean model'])/test['Agressions']))*100
test['SMAPE'] = np.mean(np.abs((test['Pred']-test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['SMAPE mean'] = np.mean(np.abs((test['Mean model'] - test['Agressions'])/((np.abs(test['Pred']) + np.abs(test['Agressions']))/2)))*100
test['MAE'] = np.mean(np.abs(test['Agressions'] - test['Pred']))
test['MAE mean'] = np.mean(np.abs(test['Agressions'] - test['Mean model']))
test['Metrica nomb'] = (test['Agressions'] < test['Pred']).sum()/len(test['Pred'])*100
test['Metrica nomb mean'] = (test['Agressions'] < test['Mean model']).sum()/len(test['Pred'])*100
test['RMSE'] = np.sqrt(mean_squared_error(test['Agressions'], test['Pred']))
test['RMSE mean'] = np.sqrt(mean_squared_error(test['Agressions'], test['Mean model']))
test['MSE'] = sqr_err(test['Agressions'], test['Pred'])
test

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,Agressions,Pred,Mean model,MAPE,MAPE mean,SMAPE,SMAPE mean,MAE,MAE mean,Metrica nomb,Metrica nomb mean,RMSE,RMSE mean,MSE
month_year,,,,,,,,,,,,,,
2022-01,4,6.30,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,5.290001
2022-01,2,3.97,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,3.880900
2022-01,6,3.35,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,7.022501
2022-01,2,3.09,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,1.188100
2022-01,1,3.09,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,4.368100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12,1,3.10,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,4.410000
2022-12,0,1.42,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,2.016400
2022-12,1,2.16,2.646104,inf,inf,88.870517,108.285525,1.975823,2.097642,58.116883,60.822511,2.82636,2.909024,1.345600


In [ ]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(deepar.tss, deepar.forecasts)
agg_metrics